# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley 2018

Due Tuesday 2pm, Feb 20, 2018

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [1]:
import pyaudio
import wave
import houndify
import houndify_login as lg

In [26]:
# create function to do record audio command and save as a wav file in format req'd for houndify
def rec_audio(fl_name, rec_time):
    
    # initialize params for pyaudio
    chunk = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    
    # do the recording
    print('Welcome to Monty!\n')
    print('Please give me a request')
    print('* begin speaking')
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    all = []
    for i in range(0, int(RATE / chunk * rec_time)):
        data = stream.read(chunk)
        all.append(data)
    print('* done recording')
    stream.close()
    p.terminate()
    
    # write to .wav file
    data = b"".join(all)
    wf = wave.open(fl_name, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

In [27]:
# given a wav audio file, use houndify to return a list of all words spoken (in order) from audio file
def audio_to_text(wav_fl, ID, KEY):
    
    # open audio file
    audio = wave.open(wav_fl)
    
    # make sure audio file is in the correct format
    if audio.getsampwidth() != 2:
        print("wrong sample width (must be 16-bit)")
    if audio.getframerate() != 8000 and audio.getframerate() != 16000:
        print("unsupported sampling frequency (must be either 8 or 16 khz)")
    if audio.getnchannels() != 1:
        print("must be single channel (mono)")

    # connect to houndify and use it to perform audio to text conversion
    client = houndify.StreamingHoundClient(ID, KEY, "test_user")
    client.setSampleRate(audio.getframerate())
    client.start()
    while True:
        samples = audio.readframes(1024)
        if len(samples) == 0: break
        if client.fill(samples): break
    result = client.finish()
    
    # return a list of all words in audio file (in order)
    return result['AllResults'][0]['RawTranscription'].split(' ')

In [28]:
# function that completely controls and runs monty
def monty(ID, KEY, fl_name = 'tmp_audio/tmp.wav', rec_time = 5):
    
    # record audio and store it as a wav file
    rec_audio(fl_name, rec_time)
    
    # get a list of words (in order) from the audio file
    request = audio_to_text(fl_name, ID, KEY)
    
    # determine what to do
    
    # detect an email request by having the words 'email', 'subject', and 'body' all appear in request
    if set(('email','body','subject')).issubset(set(request)):
        print('placeholder for email')
        
    # detect a joke request by having the words 'tell' and 'joke' in request and having request not be an email
    elif set(('tell','joke')).issubset(set(request)):
        print('placeholder for joke')
        
    # if it isn't an email or joke request, assume it is a math request
    else:
        print('placeholder for math')

In [30]:
monty(lg.ID, lg.KEY)

Welcome to Monty!

Please give me a request
* begin speaking
* done recording
placeholder for email


# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.